# Testing for E2 - using all spindles, training with all data
fistly define the UUID of the experiment to use

In [1]:
experimentId=""

-------------------------------------------------------

In [2]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## load mass

In [3]:
samplerate=200  #Should rethink this

In [4]:
annotations, signalsMetadata = loadMASSSpindles(masspath,forceSamplerate=samplerate)

In [5]:
#consider to include this in a function
minDuration=0.3
maxDuration=5
annotations=annotations[annotations.type=='spindle']
annotations=annotations[annotations.duration>minDuration]
annotations=annotations[annotations.duration<maxDuration]
annotations=annotations.reset_index(drop=True)

In [6]:
#check with EDA results (not needed)
print(1-len(annotations)/33458)

0.011297746428357902


In [7]:
annotations.head()

,type,expert,subjectId,labelerId,startTime,duration,samplerate,stopTime,startInd,stopInd
0,spindle,E1,0001,0001,888.327805,0.640579,200,888.968384,177666,177794
1,spindle,E1,0001,0001,905.758061,0.578094,200,906.336155,181152,181267
2,spindle,E1,0001,0001,917.731574,0.847603,200,918.579177,183546,183716
3,spindle,E1,0001,0001,922.078189,0.878845,200,922.957034,184416,184591
4,spindle,E1,0001,0001,939.055445,0.757767,200,939.813212,187811,187963


In [8]:
signalsMetadata.head()

,subjectId,file,channel,duration,samplerate,isOriginalSamplerate,database
0,0001,MASS_0001.pkl,C3-CLE,28956.0,200,False,MASS
1,0002,MASS_0002.pkl,C3-CLE,35016.0,200,False,MASS
2,0003,MASS_0003.pkl,C3-CLE,36760.0,200,False,MASS
3,0004,MASS_0004.pkl,C3-CLE,28004.0,200,False,MASS
4,0005,MASS_0005.pkl,C3-CLE,31244.0,200,False,MASS


## Load experiment results

In [9]:
experimentModels, featureSelection = loadExperiment(experimentId,datapath)

In [10]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,8b2ea581-6dec-4094-957e-cc1ceb7234c5,0.018639
1,0001,E1,[0001],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,80181036-469f-4bcf-95a0-68374649133a,0.020055
2,0001,E1,[0001],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,f6cafb09-5e74-4889-a306-e517e28d70fe,0.021075
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,2bf1e17a-c8bb-4166-8453-7fdff2414b10,0.018331
4,0001,E1,[0001],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,0feacbcf-4f37-402e-8fbc-4e48a2520bd4,0.019222
5,0001,E1,[0001],"[0001, 0002, 0003, 0010, 0011, 0012, 0013, 001...","[0005, 0006, 0009]",0007,29c70696-0202-4feb-bd78-3579052a4865,0.019625
6,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0007, 0010, 0011, 001...","[0002, 0017, 0013]",0009,768cb7b4-e534-49f7-b9c4-acfa7b401ffe,0.017824
7,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0009, 0012, 001...","[0017, 0001, 0011]",0010,e3d50ec6-a2f8-45fc-88f9-9af0621c2050,0.017542
8,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0009, 0010, 0012, 001...","[0002, 0018, 0007]",0011,d539204f-ac49-4811-9ed0-9b39bc65c8f3,0.015442
9,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0009, 0011, 0013, 001...","[0001, 0007, 0010]",0012,1fe9ebda-f8d4-46a7-8ec3-1c1477a12af6,0.016460


In [11]:
#we show the difference in class inbalance for the annotation criteria considered
experimentModels[['criteriumName','spindleTimeRate']].groupby('criteriumName').describe()

spindleTimeRate                                          \
                        count      mean       std       min       25%   
criteriumName                                                           
E1                       15.0  0.018463  0.001834  0.015442  0.017419   
E2                       15.0  0.056955  0.004466  0.050874  0.053849   
union                    15.0  0.058488  0.004535  0.052594  0.055095   

                                             
                    50%       75%       max  
criteriumName                                
E1             0.018331  0.019424  0.022439  
E2             0.055090  0.060190  0.065389  
union          0.056933  0.061656  0.067267

In [12]:
featureSelection

,window,characteristic,bandName,score
0,2,hjortActivity,beta2,69.0
1,1,hjortActivity,sigma,63.0
2,2,hjortActivity,beta1,42.0
3,2,hjortActivity,delta2,41.0
4,1.5,sigmaIndex,broadband,39.0
5,2,sigmaIndex,broadband,39.0
6,2,hjortActivity,theta,38.0
7,2,hjortMobility,beta1,37.0
8,2,hjortActivity,alpha,35.0
9,1,sigmaIndex,broadband,34.0


## Hyperparameter definition
this should come from a previous evaluation notebook

In [13]:
hyperClose=0.25
hyperDuration=0.3
hyperThres=0.3
hyperDepth=50

## Testing with E2 criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [14]:
experimentModels=experimentModels[experimentModels.criteriumName=='E2'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0002,E2,[0002],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,bf748cb3-0db0-4e08-b725-3f4b82679017,0.055090
1,0002,E2,[0002],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,57897053-2189-42d0-8c44-90c32b067094,0.060029
2,0002,E2,[0002],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,d9715543-4141-4773-83d3-da2830e98685,0.061873
3,0002,E2,[0002],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,dadf35ae-1f3a-43da-8303-5fff33a435c4,0.060346
4,0002,E2,[0002],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,255a0333-5bc4-45dc-aee7-89f4718866a7,0.056983
5,0002,E2,[0002],"[0001, 0002, 0003, 0010, 0011, 0012, 0013, 001...","[0005, 0006, 0009]",0007,a636bcf2-756a-4080-bb6e-a1f4cd400f46,0.060034
6,0002,E2,[0002],"[0001, 0003, 0005, 0006, 0007, 0010, 0011, 001...","[0002, 0017, 0013]",0009,5a2de0ee-a55e-4039-aef4-ee73020fbd45,0.054858
7,0002,E2,[0002],"[0002, 0003, 0005, 0006, 0007, 0009, 0012, 001...","[0017, 0001, 0011]",0010,68afbb00-3c69-45a0-96fb-c8bb1f3682c7,0.050874
8,0002,E2,[0002],"[0001, 0003, 0005, 0006, 0009, 0010, 0012, 001...","[0002, 0018, 0007]",0011,97cd3944-413e-48a8-a112-61ad4a62d585,0.053403
9,0002,E2,[0002],"[0002, 0003, 0005, 0006, 0009, 0011, 0013, 001...","[0001, 0007, 0010]",0012,b38c4b53-8f69-4715-8715-2e45fa1bf3c3,0.051788


In [15]:
#threshold for the metric by event
masterIoU=0.2

In [16]:
def getIou(coordA,coordB):
    if (coordA[1]<coordB[0])|(coordB[1]<coordA[0]): #NOT INTERSECTION
        iou=0
    else:   #INTERSECTION
        inter=np.min((coordA[1]-coordB[0],coordB[1]-coordA[0]))
        union=np.max((coordA[1]-coordB[0],coordB[1]-coordA[0]))
        iou=inter/union
    return iou

def annotationPairToMetrics(annotations,detections,thresIoU=0.2):
    #get the coords
    gtCoords=zip(annotations.startInd,annotations.stopInd)
    outCoords=zip(detections.startInd,detections.stopInd)
    #calculate the iou vector
    iouVector=np.array(list(itt.starmap(getIou,itt.product(gtCoords,outCoords))))
    #reshape to a matrix
    iouMatrix=iouVector.reshape(len(annotations),len(detections))
    #binarize
    binarized=iouMatrix>thresIoU
    #calculateF1
    outF1=(np.sum(np.max(binarized,axis=0))+np.sum(np.max(binarized,axis=1)))/(len(annotations)+len(detections))
    recall=np.sum(np.max(binarized,axis=1))/len(annotations)
    precision=np.sum(np.max(binarized,axis=0))/len(detections)
    return outF1,recall,precision

In [17]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    testSubjectId=row.test
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres
    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)
    
    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels['rawF1']=rawF1s
experimentModels['rawPrecision']=rawPrecisions
experimentModels['rawRecall']=rawRecalls

experimentModels['f1']=f1s
experimentModels['precision']=precisions
experimentModels['recall']=recalls

experimentModels['eventF1']=eventF1s
experimentModels['eventPrecision']=eventPrecisions
experimentModels['eventRecall']=eventRecalls


*************************
1 of 15
*************************
2 of 15
*************************
3 of 15
*************************
4 of 15
*************************
5 of 15
*************************
6 of 15
*************************
7 of 15
*************************
8 of 15
*************************
9 of 15
*************************
10 of 15
*************************
11 of 15
*************************
12 of 15
*************************
13 of 15
*************************
14 of 15
*************************
15 of 15


In [18]:
dumpPickle('experimentModelsTest_E2_temp.pkl',experimentModels)

In [19]:
experimentModels.columns

Index(['criteriumId', 'criteriumName', 'labelerIdList', 'train', 'val', 'test',
       'modelId', 'spindleTimeRate', 'rawF1', 'rawPrecision', 'rawRecall',
       'f1', 'precision', 'recall', 'eventF1', 'eventPrecision',
       'eventRecall'],
      dtype='object')

In [20]:
fig=px.scatter(experimentModels,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [21]:
fig=px.scatter(experimentModels,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [22]:
fig=px.scatter(experimentModels,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [23]:
fig=px.scatter(experimentModels,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [24]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0001     1.0  0.715545 NaN  0.715545  0.715545  0.715545            1.0   
0002     1.0  0.843137 NaN  0.843137  0.843137  0.843137            1.0   
0003     1.0  0.751464 NaN  0.751464  0.751464  0.751464            1.0   
0005     1.0  0.679009 NaN  0.679009  0.679009  0.679009            1.0   
0006     1.0  0.669263 NaN  0.669263  0.669263  0.669263            1.0   
0007     1.0  0.689330 NaN  0.689330  0.689330  0.689330            1.0   
0009     1.0  0.751786 NaN  0.751786  0.751786  0.751786            1.0   
0010     1.0  0.795006 NaN  0.795006  0.795006  0.795006            1.0   
0011     1.0  0.783287 NaN  0.783287  0.783287  0.783287            1.0   
0012     1.0  0.716727 NaN  0.716727  0.716727  0.716727            1.0   
0013     1.0  0.717028 NaN  0.717028  0.717028  0.717028            1.0   
0014     1.0  0.683890 NaN  0.683890  0.683890  0.683890            1.0   
0017     1.0  0.768550 NaN  0.768550  0.768550  0.768550            1.0   
0018     1.0  0.775360 NaN  0.775360  0.775360  0.775360            1.0   
0019     1.0  0.743483 NaN  0.743483  0.743483  0.743483            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0001  0.877858 NaN  0.877858  0.877858  0.877858         1.0  0.604210 NaN   
0002  0.781569 NaN  0.781569  0.781569  0.781569         1.0  0.914729 NaN   
0003  0.748333 NaN  0.748333  0.748333  0.748333         1.0  0.754622 NaN   
0005  0.529496 NaN  0.529496  0.529496  0.529496         1.0  0.942519 NaN   
0006  0.798305 NaN  0.798305  0.798305  0.798305         1.0  0.576642 NaN   
0007  0.566046 NaN  0.566046  0.566046  0.566046         1.0  0.875399 NaN   
0009  0.760883 NaN  0.760883  0.760883  0.760883         1.0  0.742962 NaN   
0010  0.760307 NaN  0.760307  0.760307  0.760307         1.0  0.832725 NaN   
0011  0.665386 NaN  0.665386  0.665386  0.665386         1.0  0.946251 NaN   
0012  0.616834 NaN  0.616834  0.616834  0.616834         1.0  0.854059 NaN   
0013  0.575472 NaN  0.575472  0.575472  0.575472         1.0  0.947368 NaN   
0014  0.526667 NaN  0.526667  0.526667  0.526667         1.0  0.967130 NaN   
0017  0.765546 NaN  0.765546  0.765546  0.765546         1.0  0.771574 NaN   
0018  0.679926 NaN  0.679926  0.679926  0.679926         1.0  0.900181 NaN   
0019  0.612015 NaN  0.612015  0.612015  0.612015         1.0  0.943756 NaN   

                                    
           min       50%       max  
test                                
0001  0.604210  0.604210  0.604210  
0002  0.914729  0.914729  0.914729  
0003  0.754622  0.754622  0.754622  
0005  0.942519  0.942519  0.942519  
0006  0.576642  0.576642  0.576642  
0007  0.875399  0.875399  0.875399  
0009  0.742962  0.742962  0.742962  
0010  0.832725  0.832725  0.832725  
0011  0.946251  0.946251  0.946251  
0012  0.854059  0.854059  0.854059  
0013  0.947368  0.947368  0.947368  
0014  0.967130  0.967130  0.967130  
0017  0.771574  0.771574  0.771574  
0018  0.900181  0.900181  0.900181  
0019  0.943756  0.943756  0.943756

In [25]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0001,0.715545,0.877858,0.604210
1,0002,0.843137,0.781569,0.914729
2,0003,0.751464,0.748333,0.754622
3,0005,0.679009,0.529496,0.942519
4,0006,0.669263,0.798305,0.576642
5,0007,0.689330,0.566046,0.875399
6,0009,0.751786,0.760883,0.742962
7,0010,0.795006,0.760307,0.832725
8,0011,0.783287,0.665386,0.946251
9,0012,0.716727,0.616834,0.854059


In [26]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              6.668000e+54
eventF1           7.388578e-01
eventPrecision    6.843096e-01
eventRecall       8.382752e-01
dtype: float64

In [27]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_16216\2931376690.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.049260
eventPrecision    0.109238
eventRecall       0.124574
dtype: float64

In [28]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels.eventPrecision,
    'event F1':experimentModels.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels.eventRecall,
    'event F1':experimentModels.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [29]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

## Restrict to only N2 stage

In [30]:
experimentModels_N2=experimentModels.copy()
experimentModels_N2.head()

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate,rawF1,rawPrecision,rawRecall,f1,precision,recall,eventF1,eventPrecision,eventRecall
0,0002,E2,[0002],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,bf748cb3-0db0-4e08-b725-3f4b82679017,0.055090,0.629691,0.833992,0.505789,0.643625,0.830075,0.525572,0.715545,0.877858,0.604210
1,0002,E2,[0002],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,57897053-2189-42d0-8c44-90c32b067094,0.060029,0.772292,0.738849,0.808905,0.772970,0.723974,0.829078,0.843137,0.781569,0.914729
2,0002,E2,[0002],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,d9715543-4141-4773-83d3-da2830e98685,0.061873,0.672400,0.690677,0.655065,0.681488,0.682511,0.680468,0.751464,0.748333,0.754622
3,0002,E2,[0002],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,dadf35ae-1f3a-43da-8303-5fff33a435c4,0.060346,0.617811,0.472894,0.890792,0.604232,0.453105,0.906623,0.679009,0.529496,0.942519
4,0002,E2,[0002],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,255a0333-5bc4-45dc-aee7-89f4718866a7,0.056983,0.589783,0.736569,0.491779,0.595829,0.724429,0.506003,0.669263,0.798305,0.576642


In [31]:
annotationsN2=pd.read_csv(datapath+'/MASS/annotations/spindlesFilteredN2.csv')
annotationsN2['samplerate']=samplerate
annotationsN2['subjectId']=annotationsN2.apply(
    lambda row: str(row.subjectId).zfill(4),axis=1)
annotationsN2['labelerId']=annotationsN2.apply(
    lambda row: str(row.labelerId).zfill(4),axis=1)
annotationsN2['stopTime']=annotationsN2.apply(
    lambda row: row.startTime+row.duration , axis=1)
annotationsN2['startInd']=annotationsN2.apply(
    lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
annotationsN2['stopInd']=annotationsN2.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)

In [32]:
## load hypnograms
stagesAnnotations=pd.read_csv(datapath+"\MASS\stages\stages.csv")
stagesAnnotations['samplerate']=samplerate
stagesAnnotations['subjectId']=stagesAnnotations.apply(
    lambda row: str(row.subjectId).zfill(4),axis=1)
stagesAnnotations['stopTime']=stagesAnnotations.apply(
    lambda row: row.startTime+row.duration , axis=1)
stagesAnnotations['startInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
stagesAnnotations['stopInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)
stagesAnnotations.head(5)

hypnograms={}
for ind, row in signalsMetadata.iterrows():
    subjectId=row.subjectId
    thisStages=stagesAnnotations[stagesAnnotations.subjectId==subjectId]
    excerptDimension=int(row.duration*row.samplerate)
    thisHypnogram=np.ones((excerptDimension,))*np.nan
    for ind_stg, row_stg in thisStages.iterrows():
        thisHypnogram[row_stg.startInd:row_stg.stopInd]=row_stg.value
    hypnograms[subjectId]=thisHypnogram   

In [33]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels_N2.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels_N2)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    testSubjectId=row.test
    testHypnogram=hypnograms[testSubjectId]
    testN2Mask=testHypnogram==2
    #Define annotations criterium
    usedAnnotations=annotationsN2[annotationsN2.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres

    #Apply mask ---------------------------------->
    rawLabels=rawLabels*testN2Mask
    #<---------------------------------------------

    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)

    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels_N2['rawF1']=rawF1s
experimentModels_N2['rawPrecision']=rawPrecisions
experimentModels_N2['rawRecall']=rawRecalls

experimentModels_N2['f1']=f1s
experimentModels_N2['precision']=precisions
experimentModels_N2['recall']=recalls

experimentModels_N2['eventF1']=eventF1s
experimentModels_N2['eventPrecision']=eventPrecisions
experimentModels_N2['eventRecall']=eventRecalls


*************************
1 of 15
*************************
2 of 15
*************************
3 of 15
*************************
4 of 15
*************************
5 of 15
*************************
6 of 15
*************************
7 of 15
*************************
8 of 15
*************************
9 of 15
*************************
10 of 15
*************************
11 of 15
*************************
12 of 15
*************************
13 of 15
*************************
14 of 15
*************************
15 of 15


In [34]:
dumpPickle('experimentModelsTest_N2_E2_temp.pkl',experimentModels_N2)

In [35]:
fig=px.scatter(experimentModels_N2,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['rawF1'], y=experimentModels_N2['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [36]:
fig=px.scatter(experimentModels_N2,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['rawF1'], y=experimentModels_N2['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [37]:
fig=px.scatter(experimentModels_N2,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [38]:
fig=px.scatter(experimentModels_N2,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [39]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0001     1.0  0.749486 NaN  0.749486  0.749486  0.749486            1.0   
0002     1.0  0.872157 NaN  0.872157  0.872157  0.872157            1.0   
0003     1.0  0.801431 NaN  0.801431  0.801431  0.801431            1.0   
0005     1.0  0.755457 NaN  0.755457  0.755457  0.755457            1.0   
0006     1.0  0.706806 NaN  0.706806  0.706806  0.706806            1.0   
0007     1.0  0.816283 NaN  0.816283  0.816283  0.816283            1.0   
0009     1.0  0.772771 NaN  0.772771  0.772771  0.772771            1.0   
0010     1.0  0.884304 NaN  0.884304  0.884304  0.884304            1.0   
0011     1.0  0.827944 NaN  0.827944  0.827944  0.827944            1.0   
0012     1.0  0.842986 NaN  0.842986  0.842986  0.842986            1.0   
0013     1.0  0.795483 NaN  0.795483  0.795483  0.795483            1.0   
0014     1.0  0.745499 NaN  0.745499  0.745499  0.745499            1.0   
0017     1.0  0.826067 NaN  0.826067  0.826067  0.826067            1.0   
0018     1.0  0.808779 NaN  0.808779  0.808779  0.808779            1.0   
0019     1.0  0.740856 NaN  0.740856  0.740856  0.740856            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0001  0.983784 NaN  0.983784  0.983784  0.983784         1.0  0.605721 NaN   
0002  0.833406 NaN  0.833406  0.833406  0.833406         1.0  0.914408 NaN   
0003  0.853333 NaN  0.853333  0.853333  0.853333         1.0  0.755481 NaN   
0005  0.628995 NaN  0.628995  0.628995  0.628995         1.0  0.943220 NaN   
0006  0.911025 NaN  0.911025  0.911025  0.911025         1.0  0.578049 NaN   
0007  0.763761 NaN  0.763761  0.763761  0.763761         1.0  0.875000 NaN   
0009  0.803869 NaN  0.803869  0.803869  0.803869         1.0  0.744172 NaN   
0010  0.943847 NaN  0.943847  0.943847  0.943847         1.0  0.832201 NaN   
0011  0.733699 NaN  0.733699  0.733699  0.733699         1.0  0.946318 NaN   
0012  0.831776 NaN  0.831776  0.831776  0.831776         1.0  0.854419 NaN   
0013  0.684521 NaN  0.684521  0.684521  0.684521         1.0  0.947253 NaN   
0014  0.604116 NaN  0.604116  0.604116  0.604116         1.0  0.967660 NaN   
0017  0.887695 NaN  0.887695  0.887695  0.887695         1.0  0.772496 NaN   
0018  0.733233 NaN  0.733233  0.733233  0.733233         1.0  0.900425 NaN   
0019  0.608752 NaN  0.608752  0.608752  0.608752         1.0  0.942913 NaN   

                                    
           min       50%       max  
test                                
0001  0.605721  0.605721  0.605721  
0002  0.914408  0.914408  0.914408  
0003  0.755481  0.755481  0.755481  
0005  0.943220  0.943220  0.943220  
0006  0.578049  0.578049  0.578049  
0007  0.875000  0.875000  0.875000  
0009  0.744172  0.744172  0.744172  
0010  0.832201  0.832201  0.832201  
0011  0.946318  0.946318  0.946318  
0012  0.854419  0.854419  0.854419  
0013  0.947253  0.947253  0.947253  
0014  0.967660  0.967660  0.967660  
0017  0.772496  0.772496  0.772496  
0018  0.900425  0.900425  0.900425  
0019  0.942913  0.942913  0.942913

In [40]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0001,0.749486,0.983784,0.605721
1,0002,0.872157,0.833406,0.914408
2,0003,0.801431,0.853333,0.755481
3,0005,0.755457,0.628995,0.943220
4,0006,0.706806,0.911025,0.578049
5,0007,0.816283,0.763761,0.875000
6,0009,0.772771,0.803869,0.744172
7,0010,0.884304,0.943847,0.832201
8,0011,0.827944,0.733699,0.946318
9,0012,0.842986,0.831776,0.854419


In [41]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              6.668000e+54
eventF1           7.964207e-01
eventPrecision    7.870542e-01
eventRecall       8.386491e-01
dtype: float64

In [42]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_16216\813125388.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.050867
eventPrecision    0.120904
eventRecall       0.124037
dtype: float64

In [43]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels_N2.eventPrecision,
    'event F1':experimentModels_N2.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels_N2.eventRecall,
    'event F1':experimentModels_N2.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [51]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

### remove outliers

In [46]:
np.setdiff1d(experimentModels_N2.test,['0006'])

array(['0001', '0002', '0003', '0005', '0007', '0009', '0010', '0011',
       '0012', '0013', '0014', '0017', '0018', '0019'], dtype=object)

In [47]:
experimentModels_N2[experimentModels_N2.test.isin(np.setdiff1d(experimentModels_N2.test,['0001','0014','0019']))][['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              1.666917e+43
eventF1           8.092057e-01
eventPrecision    8.007634e-01
eventRecall       8.386201e-01
dtype: float64

In [49]:
def annotationPairToGraph(annotations,detections,thresIoU=0.2):
    #get the coords
    gtCoords=zip(annotations.startInd,annotations.stopInd)
    outCoords=zip(detections.startInd,detections.stopInd)
    #calculate the iou vector
    iouVector=np.array(list(itt.starmap(getIou,itt.product(gtCoords,outCoords))))
    #reshape to a matrix
    iouMatrix=iouVector.reshape(len(annotations),len(detections))
    #create tables
    index0=np.apply_along_axis(np.argmax,0,iouMatrix)
    iou0=np.apply_along_axis(np.max,0,iouMatrix)
    index1=np.apply_along_axis(np.argmax,1,iouMatrix)
    iou1=np.apply_along_axis(np.max,1,iouMatrix)
    
    tableOut=pd.DataFrame({
        'indexGT':index0,
        'iou':iou0
    })

    tableGT=pd.DataFrame({
        'indexOut':index1,
        'iou':iou1
    })

    tableOut['type']='out'
    tableGT['type']='gt'
    tableOut['indexOut']=tableOut.index
    tableGT['indexGT']=tableGT.index
    #Correct external indexes of objects not overlapping
    tableOut.loc[tableOut.iou==0,'indexGT']='NA'
    tableGT.loc[tableGT.iou==0,'indexOut']='NA'
    #set tps
    tableOut['tp']=tableOut.iou>thresIoU
    tableOut['fp']=tableOut.iou<=thresIoU
    tableGT['tp']=tableGT.iou>thresIoU
    tableGT['fn']=tableGT.iou<=thresIoU
    #calculate metrics
    recall=np.sum(tableGT['tp'])/len(tableGT)
    precision=np.sum(tableOut['tp'])/len(tableOut)
    f1=(np.sum(tableGT['tp'])+np.sum(tableOut['tp']))/(len(tableGT)+len(tableOut))
    #concatenate tables
    appended=pd.concat(objs=(tableOut,tableGT),axis=0)
    #modify values
    appended['x']=appended['indexGT']
    appended['y']=appended['indexOut']
    appended.loc[((appended.type=='out')&(~ appended.tp)),'x']=-10
    appended.loc[((appended.type=='gt')&(~ appended.tp)),'y']=-10
    appended['size']=1
    appended.loc[appended.type=='out','size']=3
    #create the graph
    minTPIoU=np.min(appended[appended.tp].iou)
    fig=px.scatter(appended,x='x',y='y',color='iou',symbol='type',
    opacity=0.8,symbol_map={'out':'circle-open','gt':'circle'},size='size',
    color_continuous_scale=
        ((0.0, 'rgb(40,40,40)'),
        (0.000001, 'rgb(28,227,255)'),
        (0.14, 'rgb(56,199,255)'),
        (0.29, 'rgb(85,170,255)'),
        (0.42, 'rgb(113,142,255)'),
        (0.57, 'rgb(142,113,255)'),
        (0.71, 'rgb(170,85,255)'),
        (0.86, 'rgb(199,56,255)'),
        (1.0, 'rgb(227,28,255)')),
    range_x=(-20,len(tableGT)+10),range_y=(-20,len(tableOut)+10),
    title='by-Event evaluation summary<br><sup>F1(@IoU>'+str(thresIoU)+')='+str(round(f1,4))+' | minimum TP IoU: '+str(round(minTPIoU,4))+'</sup>',
    hover_data={'x':False,
    'y':False,
    'tp':False,
    'fp':False,
    'fn':False,
    'size':False,
    'type':False,
    'iou':':.4f', # customize hover for column of y attribute
    'indexGT':True,
    'indexOut':True
    })
    for t in fig.data:
        t.marker.line.width = 2
    fig.update_xaxes(title_text=str(len(tableGT))+' ANNOTATIONS | recall(@IoU>'+str(thresIoU)+')= '+str(round(recall,4)))
    fig.update_yaxes(title_text=str(len(tableOut))+' DETECTIONS | precision(@IoU>'+str(thresIoU)+')= '+str(round(precision,4)))
    fig.add_vline(x=-5,line_dash='dash')
    fig.add_hline(y=-5,line_dash='dash')
    #----------------------------------------------------------------------->
    # https://stackoverflow.com/questions/61827165/plotly-how-to-handle-overlapping-colorbar-and-legends
    # @vestland answer
    """ fig.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=0,
                                            ticks="outside",
                                            ticksuffix=" bills")) """
    # @bitbang answer
    fig.update_layout(legend_orientation="h")
    #<----------------------------------------------------------------------
    return fig

In [50]:
annotationPairToGraph(gtAnnotations,processedAnnotations)